In [ ]:
from manim import *
import numpy as np
from sympy import *
def inity(ox):
    x, y = symbols('x y')
    return float((solve(y**2-(x**3+109*x**2+224*x), y)[1]).subs(x,ox).doit()) #replace if needed
def equationsolver(ox,oy,nx,ny,grad,c,rflag):
    x, y = symbols('x y')
    eq1 = Eq(y-grad*x, c)
    eq2 = Eq(y**2-(x**3+109*x**2+224*x), 0) #replace if needed
    initial_guess = [(ox+nx)/2, (oy+ny)/2]
    solution = nsolve([eq1, eq2], [x, y], initial_guess,verify=False)
    solx = float(solution[0])
    soly = float(solution[1])
    if(solx == ox or solx == nx or soly == ny or soly == oy):
        if(rflag):
            solx,soly = equationsolver((ox+nx)/2,(oy+ny)/2,nx,ny,grad,c,False)
    return solx,soly
def p2line(ox,oy):
    x, y = symbols('x y')
    gradx = float(diff(x**3+109*x**2+224*x).subs(x,ox).doit()) #replace if needed
    grad=gradx/float(2*oy)
    c=float(oy-grad*ox)
    solx,soly=equationsolver(1,1,1,1,grad,c,True)
    return grad,c,solx,soly
def p3online(ox,oy,nx,ny):
    grad = float(ny-oy)/float(nx-ox)
    c = float(oy-grad*ox)
    solx,soly=equationsolver(ox,oy,nx,ny,grad,c,True)
    return grad,c,solx,soly
class EllipticCurveAnim(Scene):
    #lambda functions (replace if needed) in the format of: y**2-(x**3+109*x**2+224*x)
    #remove the y**2 term before pasting in p2line()
    def construct(self):
        timeslow = 1 #time between each point in the slow animation
        timefast = 0.1 #time between each point in the fast animation
        timepause = 5 #time buffer at start and end of video
        numslow = 3 #how many points to show in slow animation
        numfast = 26 #how many points to show in fast animation
        ox = -65 #initial point, x-coordinate
        oy = inity(ox)
        ax = Axes(x_range=[-150, 100, 10], y_range=[-700, 700, 50] #change axes yourself if needed
                  , x_length=14
                  , y_length=7.5
                  , x_axis_config={"numbers_to_include": range(-150,100,10), #here too
                                   "font_size": 18,}
                  , y_axis_config={"numbers_to_include": range(-700,700,50), #here too
                                   "font_size": 18,}
                  , tips=False)
        graph = ax.plot_implicit_curve(
            lambda x, y: y ** 2 - x ** 3 - 109*x**2 - 224*x, #replace if needed
            color=YELLOW,
        )
        num_pl = NumberPlane(x_range=[-150, 100, 10], y_range=[-700, 700, 50] #here too
                            , x_length=14
                            , y_length=7.5
                            , color=GRAY)
        num_pl.set_opacity(0)
        self.add(ax,num_pl, graph)
        self.wait(timepause)
        P = Dot(ax.coords_to_point(ox,oy), color=RED,radius=0.1)
        self.add(P)
        self.wait(timeslow)
        for i in range(numslow):
            if(i==0):
                grad,c,solx,soly=p2line(ox,oy)
            else:
                grad,c,solx,soly=p3online(ox,oy,nx,ny)
            iT = ax.plot_implicit_curve(lambda x, y: y - grad*x - c,
            color=ORANGE,
            )
            self.add(iT)
            self.wait(timeslow)
            iP = Dot(ax.coords_to_point(solx,soly),color=BLUE,radius=0.1)
            self.add(iP)
            self.remove(iT)
            self.wait(timeslow)
            iP2 = Dot(ax.coords_to_point(solx,-soly),color=RED,radius=0.1)
            self.add(iP2)
            self.remove(iP)
            self.wait(timeslow)
            nx = solx
            ny = -soly
        for j in range(numfast):
            grad,c,solx,soly=p3online(ox,oy,nx,ny)
            iT = ax.plot_implicit_curve(lambda x, y: y - grad*x - c,
            color=ORANGE,
            )
            self.add(iT)
            self.wait(timefast)
            iP = Dot(ax.coords_to_point(solx,soly),color=BLUE,radius=0.1)
            self.add(iP)
            self.remove(iT)
            self.wait(timefast)
            iP2 = Dot(ax.coords_to_point(solx,-soly),color=RED,radius=0.1)
            self.add(iP2)
            self.remove(iP)
            self.wait(timefast)
            nx = solx
            ny = -soly
        print(nx,ny)
        self.wait(timepause)